In [26]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree,metrics

from IPython.display import Image, display

# Decision Trees

## Introduction

### What are decision trees

Decision trees are a popular supervised learning algorithm used for both classification and regression tasks.

They model decisions in the form of a tree, where each internal node represents a feature, each branch represents a decision rule, and each leaf represents an outcome.

<a href="../images/from_data_to_decision_tree.bmp"><img src="../images/from_data_to_decision_tree.bmp" alt="from_data_to_decision_tree" style="height: 18em;"></a>

### Characteristics of Decision Trees

1. **Interpretability and Visualization:**
   - They are easy to interpret and visualize, allowing for transparent decision-making processes and easier communication of results to non-technical stakeholders.

2. **Applicability to Regression and Classification:**
   - Decision trees can be applied to both regression and classification tasks:
     - **Classification Trees:** Predict categorical class labels.
     - **Regression Trees:** Predict continuous numerical values.

3. **Handling of Continuous and Categorical Variables:**
   - They can handle both continuous and categorical input features. Techniques like thresholding are used for continuous variables, and category grouping for categorical variables.

4. **Hierarchical Structure and Divide-and-Conquer Strategy:**
   - Decision trees are hierarchical data structures that implement a [divide-and-conquer](https://en.wiktionary.org/wiki/divide_and_conquer) strategy. They recursively partition the dataset into subsets based on attribute values to simplify complex problems.

5. **Non-Parametric Model:**
   - Decision trees are non-parametric, meaning they make no assumptions about the underlying distribution of the data.

6. **Use of Information Theory:**
   - They extensively utilize concepts from information theory, such as entropy and information gain, to determine the optimal attribute for splitting the data at each node.

7. **Representation as If-Then Rules:**
   - They can be regarded as a set of *if-then* rules. Each path from the root to a leaf node represents a conjunction (logical AND) of conditions that lead to a specific prediction.

8. **Feature Importance Estimation:**
   - They can provide estimates of feature importance, helping in feature selection and understanding the influence of different variables.

9. **Prone to Overfitting:**
   - They can easily overfit the training data if not properly pruned or regularized. Techniques like pruning, setting minimum sample leaf sizes, or limiting tree depth are used to mitigate this.

10. **Ensemble Methods:**
    - Decision trees serve as the foundational models for ensemble methods like Random Forests and Gradient Boosting Machines, which improve predictive performance by combining multiple trees.

11. **Missing Value Handling:**
    - Some decision tree algorithms can handle missing values by assigning probabilities to different paths or using surrogate splits.


### Decision Tree Structure

<a href="../images/basic_tree_terms.png"><img src="../images/basic_tree_terms.png" alt="" style="height:14em"></a>

- **Each Node Tests an Attribute (Asks a Question):**
  - The starting node is called the **root node**.
  - Subsequent nodes that perform tests are called **internal nodes**.

- **Each Branch Corresponds to an Attribute Value:**
  - Branches represent the possible outcomes of a test and lead to the next node.

- **Each Leaf Node Assigns a Classification:**
  - Leaf nodes (terminal nodes) provide the final decision or prediction.
  - No further splitting occurs at these points.

- **Pure Node:**
  - A node where all the samples have the same class label.
  - Indicates complete homogeneity and requires no further splitting.


## How Decision Trees Work

A decision tree splits the data based on certain conditions at each node.

The goal is to create the best possible splits to maximize information gain (for classification) or minimize variance (for regression).

### *Divide and Conquer* Algorithm

1. **Divide the Dataset:**
   - The algorithm selects the *best feature (attribute)* to split the dataset, using criteria like the **Gini Index** or **Information Gain**.
   - The dataset is split into subsets based on the chosen feature's values.

2. **Conquer by Recursive Splitting:**
   - The algorithm repeats the process for each subset, selecting the best feature and splitting the data again.
   - This continues until one of the stopping conditions is met, such as:
     - The data in a subset being **pure** (all samples have the same class label).
     - No features left to split.
     - Reaching a predefined **depth limit**.
   - When a stopping condition is met, a **leaf node** is created, representing the final class label or prediction.

## Selecting the Best Feature to Split the Dataset

In decision tree algorithms, selecting the optimal feature (attribute) to split the dataset at each node is crucial for building an effective model. The goal is to choose the feature that best separates the data into homogeneous subsets - subsets where the target variable is most uniform.

### Entropy Introduction

Entropy is a fundamental concept in information theory that measures the expected amount of information produced by a random process.  
In the context of decision trees, entropy is used to determine the best way to split data to maximize information gain

**Introduced by Claude Shannon in 1948:**
  - In his seminal paper ["A Mathematical Theory of Communication"](https://en.wikipedia.org/wiki/A_Mathematical_Theory_of_Communication), Claude Shannon introduced the concept of entropy as a measure of uncertainty or information content in a communication system.

**Measurement in Bits:**
  - Entropy is typically measured in **bits** when the logarithm base 2 is used in its calculation.

**Entropy Formula**

  $$H(X) = - \sum_{i=1}^{n} P(x_i) \log_2 P(x_i)$$

   Where:
   - $H(X)$ is the entropy of the random variable X.
   - $P(x_i)$ is the probability of outcome $x_i$.
   - $n$ is the total number of possible outcomes.
   - The logarithm is base 2 (for bits).

**Example - Entropy of a Fair Coin Toss:**
  - A fair coin has two possible outcomes: heads or tails, each with a probability of $0.5$.
  - **Calculating the Entropy:**
    $$
    \begin{align*}
    E &= -\left( p_{\text{heads}} \log_2 p_{\text{heads}} + p_{\text{tails}} \log_2 p_{\text{tails}} \right) \\
      &= -\left( 0.5 \times \log_2 0.5 + 0.5 \times \log_2 0.5 \right) \\
      &= -\left( 0.5 \times (-1) + 0.5 \times (-1) \right) \\
      &= -\left( -0.5 - 0.5 \right) \\
      &= 1 \text{ bit}
    \end{align*}
    $$
    - This means each toss of a fair coin carries **1 bit** of information.

**Zero Entropy Situations:**
  - Entropy is **zero** when an outcome is certain to occur.
  - **Example:**
    - If we know a coin will definitely land on heads (probability of heads is 1), then:
      $$
      E = -\left( 1 \times \log_2 1 \right) = -\left( 1 \times 0 \right) = 0 \text{ bits}
      $$
    - Similarly, a message like "The sun will rise tomorrow" carries **0 bits** of information because it's a certain event (assuming no uncertainty).

<a href="../images/entropyfunction.png"><img src="../images/entropyfunction.png" style="height:14em"></a>

### Information Gain: The reduction in entropy after a dataset is split on a feature.

  - **Information Gain Formula:**
    $$
    IG(S, A) = E(S) - \sum_{v \in \mathrm{Values}(A)} \frac{|S_v|}{|S|} E(S_v)
    $$
    where:
    - $IG(S, A)$ is the information gain of feature $A$.
    - $S_v$ is the subset of $S$ where feature $A$ has value $v$.
    - $|S_v|$ and $|S|$ are the number of samples in $S_v$ and $S$, respectively.
    - $\mathrm{Values}(A)$ is the set of all possible values of feature $A$.

2. **Gini Impurity:**
   - Measures the probability of incorrectly classifying a randomly chosen element if it was randomly labeled according to the distribution of labels in the subset.
     - **Gini Impurity Formula:**
       $$
       G(S) = 1 - \sum_{i=1}^{c} p_i^2
       $$
       where:
       - $G(S)$ is the Gini impurity of dataset $S$.
       - $p_i$ is the proportion of samples belonging to class $i$.

   - **Gini Gain:** The reduction in Gini impurity after splitting, calculated similarly to information gain.

3. **Reduction in Variance (For Regression Trees):**
   - Used when the target variable is continuous.
   - Splits are chosen to minimize the variance within subsets.
     - **Variance Formula:**
       $$
       \mathrm{Var}(S) = \frac{1}{|S|} \sum_{i=1}^{|S|} (y_i - \bar{y})^2
       $$
       where:
       - $\mathrm{Var}(S)$ is the variance of the target variable in dataset $S$.
       - $y_i$ is the target value of the $i^\text{th}$ sample.
       - $\bar{y}$ is the mean of the target values in $S$.
       - $|S|$ is the number of samples in $S$.

### Process of Selecting the Best Feature:

1. **Calculate the Impurity of the Entire Dataset:**
   - Compute the initial impurity measure (entropy or Gini impurity) for the dataset before splitting.

2. **Evaluate Each Feature:**
   - For each feature:
     - **For Categorical Features:**
       - Split the dataset based on each possible value of the feature.
     - **For Continuous Features:**
       - Consider potential split points (thresholds) by sorting the values and evaluating splits at each threshold between adjacent values.

3. **Compute the Gain for Each Feature:**
   - Calculate the impurity of each subset resulting from the split.
   - Compute the weighted average impurity after the split.
   - Calculate the gain (reduction in impurity) achieved by the split:
     $$
     \text{Gain} = \text{Impurity before split} - \text{Weighted impurity after split}
     $$

4. **Select the Feature with the Highest Gain:**
   - Choose the feature (and threshold, if applicable) that results in the highest gain.
   - This feature is used to split the dataset at the current node.

### Considerations:

- **Handling Overfitting:**
  - Be cautious of features that create too many small subsets; this can lead to overfitting.
  - Techniques like pruning and setting minimum samples per leaf help mitigate this risk.

- **Computational Efficiency:**
  - For datasets with many features or high cardinality, computing gains can be resource-intensive.
  - Some algorithms use heuristics or random sampling to improve efficiency.

- **Bias Towards Features with Many Levels:**
  - Information gain can be biased towards features with many levels.
  - **Information Gain Ratio** adjusts for this by normalizing the gain:
    $$
    \text{Gain Ratio} = \frac{\text{Information Gain}}{\text{Split Information}}
    $$
    where **Split Information** measures the potential information generated by splitting the dataset.

### Summary:

- The selection of the best feature to split on at each node is based on maximizing the reduction in impurity.
- Common impurity measures include entropy (for information gain) and Gini impurity.
- The algorithm evaluates all possible splits and selects the one that best partitions the data into homogeneous subsets.
- This process is recursive and continues until stopping criteria are met (e.g., maximum depth, minimum samples per node, or pure nodes).

By systematically selecting features that provide the most significant reduction in impurity, the decision tree algorithm builds a model that effectively predicts the target variable.


## DecisionTreeClassifier in scikit-learn

The `DecisionTreeClassifier` is a class in the scikit-learn library used for classification tasks based on decision trees.

It allows for easy implementation of decision tree models, offering control over the tree's depth, splitting criteria, and other hyperparameters.

```python
    from sklearn.tree import DecisionTreeClassifier

    # Create a DecisionTreeClassifier object
    clf = DecisionTreeClassifier(criterion='gini', max_depth=3)

    # Fit the model to training data
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)
```

## Notes on Feature Encoding 

Theoretically, for decision trees, it is not necessary to scale or normalize features, as the algorithm does not rely on the distance between data points.

However, in scikit-learn the DecisionTreeClassifier requires categorical features to be encoded numerically. Ussually encoding is done with Label Encoding, because its effcient.

Decision Trees, unlike linear models, do not assume any order or distance between the values of categorical features. For instance, in the case of Outlook with values "Sunny", "Rain", and "Overcast", even though we use numerical labels (e.g., 0, 1, 2), the tree will not interpret these as having a natural order. It will split the feature purely based on the information gain (or Gini/entropy), treating each value separately, just like it would for distinct categories.

## Pros and Cons of Decision Trees Algorithm

### Pros:

1. **Easily interpretable (not a "Black-Box")**  
   Decision Trees are highly interpretable, allowing users to understand and explain how the model reaches any decision.

1. **Requires minimal data preparation**  
   Unlike many algorithms, Decision Trees require little to no data preprocessing, such as normalization or scaling.

1. **Handles missing data and irrelevant attributes**  
   Decision Trees can work effectively even with missing data or irrelevant features, where irrelevant attributes may have a gain of zero.

1. **Can model problems with multiple outputs**  
   Decision Trees can handle and predict multiple output variables simultaneously.

1. **Low memory footprint**  
   After pruning, Decision Trees are highly compact and require minimal memory.

1. **Fast prediction phase (O(treeDepth))**  
   While training may take time, prediction using Decision Trees is extremely fast, proportional to the depth of the tree.

1. **Non-parametric**  
   Decision Trees do not assume a predefined distribution for the data, making them suitable for various types of data distributions. More on [non-parametric models](https://machinelearningmastery.com/parametric-and-nonparametric-machine-learning-algorithms/).

### Cons:

1. **Prone to overfitting**  
   Decision Trees can easily overfit the training data, especially if the tree is too deep, leading to poor generalization on new data.

1. **Unstable (high variance)**  
   Small changes in the data can result in a completely different tree structure, making Decision Trees sensitive to variations in the dataset.

1. **Biased with imbalanced data**  
   Decision Trees may favor classes with more instances, leading to biased predictions if the dataset is imbalanced.

1. **Inefficient with large datasets**  
   As the dataset size grows, Decision Trees can become computationally expensive to train, especially if the depth is not constrained.

1. **Limited to axis-aligned splits**  
   Decision Trees only make splits that are parallel to the feature axes, which may not capture more complex decision boundaries well.

1. **Requires pruning for optimal performance**  
   Without pruning, Decision Trees can grow very large, which negatively impacts performance and interpretability.

1. **Poor extrapolation**  
   Decision Trees are not good at extrapolating beyond the range of the training data, which can limit their predictive power in some cases.


## Applications of Decision Trees:

1. **Classification tasks**  
   Decision Trees are widely used for classifying objects, events, or people into predefined categories, such as spam detection, medical diagnosis, and customer segmentation.

1. **Regression tasks**  
   Decision Trees can be applied for predicting continuous values, such as house prices, stock market trends, and demand forecasting.

1. **Feature selection**  
   Decision Trees help identify the most important features in a dataset by calculating feature importance through information gain or Gini impurity.

### Areas of application:

1. **Credit scoring**  
   Financial institutions use Decision Trees to evaluate the risk of loan applicants and predict whether a customer is likely to default.

1. **Fraud detection**  
   Decision Trees are employed in detecting fraudulent activities in transactions by learning patterns and anomalies in the data.

1. **Customer churn prediction**  
   Companies use Decision Trees to predict customer churn by analyzing customer behavior and identifying factors that lead to customer loss.

1. **Medical diagnosis and decision making**  
   In healthcare, Decision Trees assist in diagnosing diseases based on symptoms and test results, as well as making treatment recommendations.

1. **Market basket analysis**  
   Decision Trees help in understanding customer purchasing behavior by analyzing patterns in transactions, enabling targeted marketing strategies.


## Example: calculate Information Gain

We will build a Decision Tree using the following dataset to predict whether we can "Play" based on the features **Outlook**, **Humidity**, and **Wind**.

| Outlook  | Humidity | Wind   | Play |
|----------|----------|--------|------|
| Sunny    | High     | Weak   | No   |
| Sunny    | High     | Strong | No   |
| Overcast | High     | Weak   | Yes  |
| Rain     | High     | Weak   | Yes  |
| Rain     | Normal   | Weak   | Yes  |
| Rain     | Normal   | Strong | No   |
| Overcast | Normal   | Strong | Yes  |
| Sunny    | High     | Weak   | No   |
| Sunny    | Normal   | Weak   | Yes  |
| Rain     | Normal   | Weak   | Yes  |
| Sunny    | Normal   | Strong | Yes  |
| Overcast | High     | Strong | Yes  |
| Overcast | Normal   | Weak   | Yes  |
| Rain     | High     | Strong | No   |


1. **Calculate the entropy of the target (Play) variable:**
   - There are 9 "Yes" and 5 "No" instances.
   - The entropy is calculated as:

   $$
   H(Play) = - \left( \frac{9}{14} \log_2 \frac{9}{14} \right) - \left( \frac{5}{14} \log_2 \frac{5}{14} \right) = 0.940
   $$

2. **Calculate Information Gain for each feature:**
   - We calculate the Information Gain for **Outlook**, **Humidity**, and **Wind** by splitting on each feature and calculating the weighted average entropy for the subsets.
   
   We split **Outlook** into three categories: Sunny, Overcast, and Rain. The entropy is calculated for each subset, and then we calculate the weighted average entropy for **Outlook**.

   1. For **Sunny** (5 instances: 3 No, 2 Yes):

      $$
      H(Sunny) = - \left( \frac{3}{5} \log_2 \frac{3}{5} \right) - \left( \frac{2}{5} \log_2 \frac{2}{5} \right) = 0.971
      $$

   2. For **Overcast** (4 instances: 0 No, 4 Yes):

      $$
      H(Overcast) = - \left( \frac{4}{4} \log_2 \frac{4}{4} \right) = 0
      $$

   3. For **Rain** (5 instances: 3 Yes, 2 No):

      $$
      H(Rain) = - \left( \frac{3}{5} \log_2 \frac{3}{5} \right) - \left( \frac{2}{5} \log_2 \frac{2}{5} \right) = 0.971
      $$

   Now, calculate the weighted average entropy for **Outlook**:

      $$
      H(Outlook) = \frac{5}{14} \cdot H(Sunny) + \frac{4}{14} \cdot H(Overcast) + \frac{5}{14} \cdot H(Rain)
      $$

   Substituting the values:

      $$
      H(Outlook) = \frac{5}{14} \cdot 0.971 + \frac{4}{14} \cdot 0 + \frac{5}{14} \cdot 0.971 = 0.693
      $$

   Thus, the entropy for **Outlook** is approximately 0.693.


   - Information Gain for **Outlook**:

   $$
   IG(Outlook) = 0.940 - 0.693 = 0.247
   $$

   **Humidity**:
   - Split into High (3 Yes, 4 No), Normal (6 Yes, 1 No).
   - The entropy for **Humidity** is:

   $$
   H(Humidity) = \frac{7}{14}(0.985) + \frac{7}{14}(0.592) = 0.788
   $$

   - Information Gain for **Humidity**:

   $$
   IG(Humidity) = 0.940 - 0.788 = 0.152
   $$

   **Wind**:
   - Split into Weak (6 Yes, 2 No), Strong (3 Yes, 3 No).
   - The entropy for **Wind** is:

   $$
   H(Wind) = \frac{8}{14}(0.811) + \frac{6}{14}(1) = 0.892
   $$

   - Information Gain for **Wind**:

   $$
   IG(Wind) = 0.940 - 0.892 = 0.048
   $$

3. **Choose the feature with the highest Information Gain:**
   - **Outlook** has the highest information gain (0.247), so we split on **Outlook**.

4. **Split the data based on Outlook:**
   - For **Overcast**, all instances are "Yes", so this becomes a leaf node labeled "Yes".
   - For **Rain** and **Sunny**, further splits are needed.

5. **Repeat the process for the remaining splits** using **Humidity** and **Wind** to build the complete tree.


## Hands On: Will John play Tennis?
 
### Data insights

In [29]:
df = pd.read_csv('./play_tennis_data.csv')
df.sort_values(by=['Outlook','Humidity','Wind'], inplace=True)
df.head()

,Outlook,Humidity,Wind,Play
11,Overcast,High,Strong,Yes
2,Overcast,High,Weak,Yes
6,Overcast,Normal,Strong,Yes
12,Overcast,Normal,Weak,Yes
13,Rain,High,Strong,No


In [30]:
def highlight_play(row):
    '''Function to apply color based on the value in the 'Play' column'''
    color = 'red' if row['Play'] == 'No' else 'green'
    return [f'color:{color}'] * len(row)

# Show data with colors
df.style.apply(highlight_play, axis=1)


,Outlook,Humidity,Wind,Play
11,Overcast,High,Strong,Yes
2,Overcast,High,Weak,Yes
6,Overcast,Normal,Strong,Yes
12,Overcast,Normal,Weak,Yes
13,Rain,High,Strong,No
3,Rain,High,Weak,Yes
5,Rain,Normal,Strong,No
4,Rain,Normal,Weak,Yes
9,Rain,Normal,Weak,Yes
1,Sunny,High,Strong,No


### Prepare data

#### Categorical attributes to values

### Split to train/test sets

### Train the model

### Make predictions

### Viualize decision bounaries

In [ ]:
# visualize the model's decision regions
from utils import plot_decision

X_combined = np.vstack((X_train, X_test))
y_combined = np.hstack((y_train, y_test))
plot_decision(X=X_combined, y=y_combined, classifier=fitted)
plt.legend(loc='upper left')
plt.show()

### Evaluate the model performance